In [1]:
# magic    
get_ipython().run_line_magic('matplotlib', 'inline')
# run files
get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/import_libs.py')
get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/Connecting_.py')
get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/my_script.py')
get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/Date_gen.py')
get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/xx.py')

In [2]:
# В НАЧАЛЕ СКРИПТА АВТОЗАПУСКА
# logs crontab АВТОЗАПУСК
start_datetime = datetime.datetime.today()
name_file = r'[АВТОЗАПУСК] Источники для BI (Продажи_)-last2month.py'
path_ipy = r"/opt/anaconda3/envs/jupyter_env/bin/ipython" 
path_file = fr"/home/adorofeev/jhub-analytics/'[adorofeev] Дорофеев Артём Александрович'/'Продажи'/'{name_file}'" 
log_info = []

In [33]:
path_SQL_sales = glob(abs_path + r'**/' + r"*АВТОЗАПУСК] Источники для BI (Продажи_).sql", recursive=True)[2]
print(f'path_SQL_sales - {path_SQL_sales}')

path_SQL_sales - /home/adorofeev/jhub-analytics/[adorofeev] Дорофеев Артём Александрович/Продажи/ [АВТОЗАПУСК] Источники для BI (Продажи_).sql


In [28]:
date_df_start = df_date_m['start'][0]
date_df_end = df_date_m.iloc[-1, -1]

path_SQL_sales - /home/adorofeev/[adorofeev] Дорофеев Артём Александрович/Продажи/[АВТОЗАПУСК] Источники для BI (Продажи_).sql


In [13]:
# public_link = '''https://nextcloud.e2e4.ru/apps/files/?dir=/Public/Отчеты%20(выгрузка%20jupyter%20notebooks)&fileid=2018892'''
AmazonS3_folder_Public = r'/Public/Отчеты (выгрузка jupyter notebooks)'
AmazonS3_path_to_file = r'/Источники для BI/Продажи_/'
# print('AmazonS3_file_name - ', AmazonS3_file_name, '/nAmazonS3_full_path - ', AmazonS3_full_path)
Amazon_owncloud = owncloud.Client('https://nextcloud.e2e4.ru')
Amazon_owncloud.login(loginAtlassianjhubAdmin, passwordAtlassianjhubAdmin)
### Amazon_owncloud.mkdir('testdir')

In [23]:
i = df_date_m.iloc[-2:-1,:].values[0]
i

array(['2023-07-01', '2023-07-31'], dtype=object)

In [24]:
AmazonS3_file_name = fr'(Тест id_комплектов)Источники для BI (Продажи) {i[0]} - {i[1]}' # ({getpass.getuser()})
AmazonS3_full_path = AmazonS3_folder_Public + AmazonS3_path_to_file + AmazonS3_file_name

print(f'{i[0]} || {i[1]}')

2023-07-01 || 2023-07-31


In [34]:
with open( path_SQL_sales, "r", encoding = "utf-8" ) as sql_file_sales:
#       коннектимся 
    with psycopg2.connect(dbname=database, user=user, password=password, host=host) as cnxn:
#           подставляем даты
        print('подставляем даты, выгружаем')
        df_fon = pd.read_sql_query(sql_file_sales.read().replace('DATE_START_replce', f'{i[0]}').replace('DATE_END_replce', f'{i[1]}'), cnxn) 
        print('✔ .read_sql_query')

подставляем даты, выгружаем
✔ .read_sql_query


In [37]:
log_info.append( (i, df_fon.shape) )
if df_fon.shape[0] != 0:
#   to_excel
    df_fon.to_excel(temp_tmp + AmazonS3_file_name + '.xlsx',  index=False, encoding='cp-1251') 
    print('✔ .to_excel')
#   to_csv 
#     df_fon.to_csv(path_OUT_dbDATA + r'Фоновый отчет по выручке (по дате продажи) с возвратами/' \
#         + fr'Источники для BI (Продажи) { i[0] } { i[1] } .csv', sep='☺',  index=False) # cp-1251 encoding='UTF-8', engine='xlsxwriter'
#   .to_pickle
    df_fon.to_pickle(temp_tmp + AmazonS3_file_name + '.pkl', compression='zip') 
    print('✔ .to_pickle')

    Amazon_owncloud.put_file( # полный путь и там и там!!! 
                AmazonS3_full_path + '.xlsx', temp_tmp + AmazonS3_file_name + '.xlsx')
    print( f'✔ Файл записан в облако\n {AmazonS3_full_path}.xlsx')  

    Amazon_owncloud.put_file( # полный путь и там и там!!! 
                AmazonS3_full_path + '.pkl', temp_tmp + AmazonS3_file_name + '.pkl'
                ) 
    print( f'✔ Файл записан в облако\n {AmazonS3_full_path}.pkl') 

    os.remove(temp_tmp + AmazonS3_file_name + '.xlsx')
    os.remove(temp_tmp + AmazonS3_file_name + '.pkl')
    print('✔ os.remove')
#     print(df_fon) 
    print(f"{datasize.DataSize(sys.getsizeof(df_fon)):MiB}", '(MiB)\n\n' )

else:
    print( df_fon, '-- df.shape =', df_fon.shape)

✔ .to_excel
✔ .to_pickle
✔ Файл записан в облако
 /Public/Отчеты (выгрузка jupyter notebooks)/Источники для BI/Продажи_/(Тест id_комплектов)Источники для BI (Продажи) 2023-07-01 - 2023-07-31.xlsx
✔ Файл записан в облако
 /Public/Отчеты (выгрузка jupyter notebooks)/Источники для BI/Продажи_/(Тест id_комплектов)Источники для BI (Продажи) 2023-07-01 - 2023-07-31.pkl
✔ os.remove
356.8668260574341MiB (MiB)




In [ ]:
## идем по датам, коннектимся, читаем sql file, подставляем даты
df_fon = pd.DataFrame()
for i in df_date_m.iloc[-2:,:].values:
    AmazonS3_file_name = fr'(Тест id_комплектов)Источники для BI (Продажи) {i[0]} - {i[1]}' # ({getpass.getuser()})
    AmazonS3_full_path = AmazonS3_folder_Public + AmazonS3_path_to_file + AmazonS3_file_name

    print(f'{i[0]} || {i[1]}')
#   читаем sql file
    with open( path_SQL_sales, "r", encoding = "utf-8" ) as sql_file_sales:
#       коннектимся 
        with psycopg2.connect(dbname=database, user=user, password=password, host=host) as cnxn:
#           подставляем даты
            print('подставляем даты, выгружаем')
            df_fon = pd.read_sql_query(sql_file_sales.read().replace('DATE_START_replce', f'{i[0]}').replace('DATE_END_replce', f'{i[1]}'), cnxn) 
            print('✔ .read_sql_query')
    
#   !! отсекаем товары на сегодня потому что грузится ночью до указанной даты а надо до всчерашнего конца дня ИСКЛЮЧАЯ СЕГОНЯ НОЧЬ!!  
    df_fon = df_fon[df_fon['Дата отгрузки товара'].astype(str) != pd.Timestamp.now().strftime( '%Y-%m-%d' )]
            
    log_info.append( (i, df_fon.shape) )
    if df_fon.shape[0] != 0:
    #   to_excel
        df_fon.to_excel(temp_tmp + AmazonS3_file_name + '.xlsx',  index=False, encoding='cp-1251') 
        print('✔ .to_excel')
    #   to_csv 
    #     df_fon.to_csv(path_OUT_dbDATA + r'Фоновый отчет по выручке (по дате продажи) с возвратами/' \
    #         + fr'Источники для BI (Продажи) { i[0] } { i[1] } .csv', sep='☺',  index=False) # cp-1251 encoding='UTF-8', engine='xlsxwriter'
    #   .to_pickle
        df_fon.to_pickle(temp_tmp + AmazonS3_file_name + '.pkl', compression='zip') 
        print('✔ .to_pickle')

        Amazon_owncloud.put_file( # полный путь и там и там!!! 
                    AmazonS3_full_path + '.xlsx', temp_tmp + AmazonS3_file_name + '.xlsx')
        print( f'✔ Файл записан в облако\n {AmazonS3_full_path}.xlsx')  

        Amazon_owncloud.put_file( # полный путь и там и там!!! 
                    AmazonS3_full_path + '.pkl', temp_tmp + AmazonS3_file_name + '.pkl'
                    ) 
        print( f'✔ Файл записан в облако\n {AmazonS3_full_path}.pkl') 

        os.remove(temp_tmp + AmazonS3_file_name + '.xlsx')
        os.remove(temp_tmp + AmazonS3_file_name + '.pkl')
        print('✔ os.remove')
    #     print(df_fon) 
        print(f"{datasize.DataSize(sys.getsizeof(df_fon)):MiB}", '(MiB)\n\n' )

    else:
        print( df_fon, '-- df.shape =', df_fon.shape)